In [9]:
import random
import threading
import time
from sqlalchemy import create_engine, Column, Integer, String, DateTime, func
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy.exc import NoResultFound


In [10]:
#---- this seems to work!
import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from contextlib import contextmanager

from sqlalchemy import create_engine, Column, Integer, String, Boolean, DateTime
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy.exc import NoResultFound
from sqlalchemy.sql import text

# --- Setup the SQLite database & SQLAlchemy ORM ---

# Using a file-based SQLite database and allow multithreaded access:
#engine = create_engine("sqlite:///test9.db", connect_args={"check_same_thread": False})
engine = create_engine('postgresql://postgres@localhost:5333/test_tile')
SessionLocal = sessionmaker(bind=engine)

Base = declarative_base()

class Tile(Base):
    __tablename__ = 'tiles'
    id = Column(Integer, primary_key=True)
    annotation_class_id = Column(Integer, default=1)  # for demo, all tiles use 1
    hasgt = Column(Boolean, default=True)
    datetime = Column(DateTime, default=datetime.datetime.utcnow)
    status = Column(String, default="pending")  # can be "pending", "in_progress", etc.
    worker_id = Column(Integer, nullable=True)    # which worker claimed it

# Drop and recreate the table (for demo purposes)
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

In [11]:
# --- Populate the database with some tiles ---
with SessionLocal() as session:
    tiles = []
    for i in range(10):  # create 50 tiles
        tile = Tile(
            annotation_class_id=1,
            hasgt=True,
            # Newer tiles have a more recent datetime:
            datetime=datetime.datetime.utcnow() - datetime.timedelta(seconds=i)
        )
        tiles.append(tile)
    session.add_all(tiles)
    session.commit()

In [12]:
# --- Utility to get a session ---
@contextmanager
def get_session():
    session = SessionLocal()
    try:
        yield session
        session.commit()
    except Exception as e:
        session.rollback()
        raise e
    finally:
        session.close()

In [13]:
# -- works with sqlite but not postgis

# # --- The tile-claiming function ---
# def getWorkersTile(worker_id):
#     """
#     Atomically retrieves and marks a tile as 'in_progress' so no two workers claim the same tile.
    
#     Since SQLite does not support row-level locking (FOR UPDATE SKIP LOCKED), we use an atomic update query.
#     """
#     with get_session() as db_session:
#         # Atomically select and update a tile in one query
#         tile = db_session.execute(
#             text("""
#                 UPDATE tiles 
#                 SET status = 'in_progress', worker_id = :worker_id 
#                 WHERE id = (
#                     SELECT id FROM tiles 
#                     WHERE annotation_class_id = 1 
#                     AND hasgt = True 
#                     AND status = 'pending' 
#                     ORDER BY datetime DESC 
#                     LIMIT 1
#                 )
#                 RETURNING id, annotation_class_id, hasgt, datetime, status, worker_id
#             """),
#             {"worker_id": worker_id}
#         ).fetchone()

#         if tile:
#             return f"Worker {worker_id} claimed Tile {tile[0]}"
#         else:
#             return f"Worker {worker_id} found no tile"

In [14]:
# #--- works postgis
# # --- The tile-claiming function ---
# def getWorkersTile(worker_id):
#     """
#     Atomically retrieves and marks a tile as 'in_progress' so no two workers claim the same tile.
    
#     In PostgreSQL, we use `FOR UPDATE SKIP LOCKED` to ensure that a tile is locked and
#     unavailable for other workers until the current worker commits the transaction.
#     """
#     with get_session() as db_session:
#         # Atomically select and update a tile in one query
#         tile = db_session.execute(
#             text("""
#                 WITH selected_tile AS (
#                     SELECT id FROM tiles 
#                     WHERE annotation_class_id = 1 
#                     AND hasgt = True 
#                     AND status = 'pending'
#                     ORDER BY datetime DESC 
#                     LIMIT 1 
#                     FOR UPDATE SKIP LOCKED
#                 )
#                 UPDATE tiles 
#                 SET status = 'in_progress', worker_id = :worker_id 
#                 WHERE id = (SELECT id FROM selected_tile)
#                 RETURNING id, annotation_class_id, hasgt, datetime, status, worker_id
#             """),
#             {"worker_id": worker_id}
#         ).fetchone()

#         if tile:
#             return f"Worker {worker_id} claimed Tile {tile[0]}"
#         else:
#             return f"Worker {worker_id} found no tile"


In [15]:
#--- doesn't work sqlite - works postgis
from sqlalchemy.orm import aliased
from sqlalchemy import func

# --- The tile-claiming function ---
def getWorkersTile(worker_id):
    """
    Atomically retrieves and marks a tile as 'in_progress' so no two workers claim the same tile.
    
    This version uses SQLAlchemy ORM with row-level locking to ensure atomicity.
    """
    with get_session() as db_session:
        # Get the Tile class and the query for the tile we're interested in
        tile_query = db_session.query(Tile).filter(
            Tile.annotation_class_id == 1,
            Tile.hasgt == True,
            Tile.status == 'pending'
        ).order_by(Tile.datetime.desc())

        # Lock the selected tile for the current transaction and ensure no other worker can claim it
        tile_query = tile_query.with_for_update(skip_locked=True)

        # Select the first tile
        tile = tile_query.first()

        if tile:
            # Update the selected tile's status and worker_id
            tile.status = 'in_progress'
            tile.worker_id = worker_id
            db_session.commit()

            return f"Worker {worker_id} claimed Tile {tile.id}"
        else:
            return f"Worker {worker_id} found no tile"


In [16]:
# --- Worker function ---
def worker_function(worker_id):
    result = getWorkersTile(worker_id)
    print(result)
    return result

In [17]:
# --- Main function: spawn many workers concurrently ---
def main():
    num_workers = 200  # simulate an aggressive scenario with 200 concurrent workers
    results = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(worker_function, worker_id) for worker_id in range(num_workers)]
        for future in as_completed(futures):
            results.append(future.result())
    
    print("\nSummary:")
    claimed = [r for r in results if "claimed Tile" in r]
    for res in results:
        print(res)
    print(f"\nTotal claimed tiles: {len(claimed)}")

if __name__ == "__main__":
    main()

Worker 0 claimed Tile 1
Worker 2 claimed Tile 3
Worker 4 claimed Tile 7
Worker 5 claimed Tile 5
Worker 3 claimed Tile 2
Worker 9 claimed Tile 6
Worker 11 found no tile
Worker 10 found no tile
Worker 6 claimed Tile 9
Worker 15 found no tile
Worker 14 found no tile
Worker 1 claimed Tile 4
Worker 12 found no tile
Worker 16 found no tile
Worker 7 claimed Tile 10
Worker 17 found no tile
Worker 13 found no tile
Worker 8 claimed Tile 8
Worker 19 found no tile
Worker 22 found no tile
Worker 24 found no tile
Worker 26 found no tile
Worker 20 found no tile
Worker 18 found no tile
Worker 23 found no tile
Worker 29 found no tile
Worker 21 found no tile
Worker 31 found no tile
Worker 28 found no tile
Worker 35 found no tile
Worker 30 found no tile
Worker 25 found no tile
Worker 33 found no tile
Worker 27 found no tile
Worker 40 found no tile
Worker 39 found no tile
Worker 41 found no tile
Worker 37 found no tile
Worker 34 found no tile
Worker 44 found no tile
Worker 36 found no tileWorker 38 found 